In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from gerrychain import Graph, GeographicPartition, Election, MarkovChain, tree, constraints
from functools import partial
from gerrychain.updaters import Tally, cut_edges
from gerrychain.metrics import polsby_popper
from gerrychain.proposals import recom
import os
from gerrychain.accept import always_accept

from combination import *
from candidate import Candidate
from ballot import Ballot

In [2]:
PROCESSED_SHAPEFILE_PATH = os.path.join(os.getcwd())
UT_PATH = os.path.join(PROCESSED_SHAPEFILE_PATH, "utah", "UT_precincts.shp")
EXPORT_PATH = os.path.join(PROCESSED_SHAPEFILE_PATH, "utah")

ITERATIONS = 1
SAFE_SEAT_THRESHOLD = 0.66
MINORITY_POP_THRESHOLD = RACE_WINNER_THRESHOLD = 0.5
TOTAL_DISTRICTS = 4
ONE = 1

In [3]:
ut_graph = Graph.from_file(UT_PATH, ignore_errors=True)

/Users/alexisperezmendoza/Desktop/python-mggg/mggg/lib/python3.10/site-packages/gerrychain/graph/adjacency.py:21: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  geometries[i].id = i
/Users/alexisperezmendoza/Desktop/python-mggg/mggg/lib/python3.10/site-packages/gerrychain/graph/adjacency.py:23: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  tree = STRtree(geometries)
/Users/alexisperezmendoza/Desktop/python-mggg/mggg/lib/python3.10/site-packages/gerrychain/graph/adjacency.py:66: UserWarning: Found overlaps among the given polygons. Indices of overlaps: {(1805, 1814), (444, 445), (95, 377), (55, 1396), (136, 149), (716, 724), (719, 720), (716, 733), (930, 931), (1518, 1519), (719, 729), (787, 1022), (40, 41), (628, 629), (1129, 1149), (1690, 1860), (956, 1087), (1884, 2000), (1968, 1975), (1569, 1570), (929, 962), (124, 1370), (963, 964)

In [4]:
total_population = sum(ut_graph.nodes[node]["TOTPOP"] for node in ut_graph.nodes)


In [5]:
partition_1 = tree.recursive_tree_part(ut_graph, [1], total_population, "TOTPOP", 0.01, 1)

ut_election = Election("CongressionalRace" , {"Democratic": "SEN16D", "Republican": "SEN16R"})

In [6]:
new_partition = GeographicPartition(ut_graph, assignment=partition_1, updaters={
        "population": Tally("TOTPOP", alias="population"),
        "white": Tally("NH_WHITE", alias="white"),
        "black": Tally("NH_BLACK", alias="black"),
        "asian": Tally("NH_ASIAN", alias="asian"),
        "hisp": Tally("HISP", alias="hisp"),
        "other": Tally("NH_OTHER", alias="other"),
        "cut_edges": cut_edges,
        "CongressionalRace": ut_election
    })

In [7]:
ideal_pop = total_population / ONE
proposal = partial(recom, pop_col="TOTPOP", pop_target=ideal_pop, epsilon=.02, node_repeats=2)
compactness = constraints.UpperBound(
    lambda p: len(p["cut_edges"]),
    2*len(new_partition["cut_edges"])
)
pop_constraint = constraints.within_percent_of_ideal_population(new_partition, 0.3)


In [8]:
chain1 = MarkovChain(
    proposal=proposal,
    constraints=[
    ],
    accept=always_accept,
    initial_state=new_partition,
    total_steps=ITERATIONS
)



In [9]:
import random 

ballot_map = {}
vote_map = {}
all_candidates_dem = [Candidate("Whipper, Snapper", "Democratic"), Candidate("Williams, Abigail", "Democratic"),
                     Candidate("Rob Crok", "Democratic"), Candidate("Kennedy, Robert J.", "Democratic")]
                     
all_candidates_rep = [Candidate("Stewart, Chris", "Republican"), Candidate("Owens, Burgess", "Republican"),
                      Candidate("Moore, Blake D.", "Republican"), Candidate("Curtis, John R.", "Republican")]


for step in chain1:
    for part in step.parts:
        ballots = []

        total_reps = round((step["population"][part] / total_population) * TOTAL_DISTRICTS)
        dem_candidates = []
        rep_candidates = []
        for i in range(total_reps):
            dem_candidates.append(all_candidates_dem.pop())
            rep_candidates.append(all_candidates_rep.pop())

        rep_votes = 0
        dem_votes = 0
        for node in step.parts[part]:
            random.shuffle(dem_candidates)
            random.shuffle(rep_candidates)
            # Create ballots for the Republican and Democratic candidates
            ballots += [Ballot([rep for rep in rep_candidates]) for _ in range(round(int(ut_graph.nodes[node]["SEN16R"])))]
            ballots += [Ballot([dem for dem in dem_candidates]) for _ in range(round(int(ut_graph.nodes[node]["SEN16D"])))]
            rep_votes += round(int(ut_graph.nodes[node]["SEN16R"]))
            dem_votes += round(int(ut_graph.nodes[node]["SEN16D"]))
        ballot_map[part] = ballots
        vote_map[part] = (dem_votes + rep_votes, total_reps)


In [10]:
from collections import Counter

election_results = {}
initial_votes = {}

for district in ballot_map:
    ballots = ballot_map[district]
    winning_threshold = vote_map[district][0] / (1+vote_map[district][1])
    vote_counts = Counter()

    remaining_candidates = []
    eliminated_candidates = []
    winners = []
    winner_count = 0
    candidates_dict = {}
    rounds = 0

    for ballot in ballots:
        first_candidate = ballot.rankings[0]
        vote_counts[first_candidate] += ballot.weight

    remaining_candidates = vote_counts.most_common(vote_map[district][1] * 2)
    remaining_candidates.sort(key=lambda x: x[1], reverse=True)
    initial_votes[district] = remaining_candidates

    while(True):
        remaining_candidates = vote_counts.most_common((vote_map[district][1] * 2) - rounds - winner_count)
        remaining_candidates.sort(key=lambda x: x[1], reverse=True)

        if len(remaining_candidates)-1 + len(winners) == vote_map[district][1]:
            winners += remaining_candidates
            eliminated_candidates.append(winners.pop())
            break

        # SURPLUS ROUND
        first_place = remaining_candidates[0]
        if first_place[1] >= winning_threshold:
            remaining_candidates.remove(first_place)
            surplus = first_place[1] - winning_threshold

            sp_count = 0
            for ballot in ballots:
                if ballot.rankings[0] == first_place[0]:
                    ballot.rankings.remove(ballot.rankings[0])
                    ballot.set_weight(ballot.get_weight() * .8)
                    vote_counts[ballot.rankings[0]] = round(ballot.weight + vote_counts[ballot.rankings[0]], 4)
                    sp_count += 1
                    if sp_count >= surplus:
                        break
            for ballot in ballots:
                if ballot.rankings[0] == first_place[0]:
                    vote_counts[ballot.rankings[0]] -= 1

            first_place = (first_place[0], winning_threshold)
            winners.append(first_place) 
            rounds +=1
            winner_count += 1
            continue

        # ELIMINATION ROUND
        eliminated = remaining_candidates[(vote_map[district][1] * 2) - rounds-1]
        if len(remaining_candidates)-1 + len(winners) == vote_map[district][1]:
            remaining_candidates.remove(eliminated)
            winners += remaining_candidates
            eliminated_candidates.append(eliminated)
            break
        for ballot in ballots:
            if ballot.rankings[0] == eliminated[0]:
                ballot.rankings.remove(ballot.rankings[0])
                ballot.set_weight(ballot.get_weight() * .8)
                vote_counts[ballot.rankings[0]] = round(ballot.weight + vote_counts[ballot.rankings[0]], 4)
                
        for ballot in ballots:
            if ballot.rankings[0] == eliminated[0]:
                vote_counts[ballot.rankings[0]] -= 1

        eliminated_candidates.append(eliminated)
        rounds += 1
        election_results[district] = {"Winners": winners, "Eliminated": eliminated_candidates, "Initial Votes": initial_votes[district]}


In [11]:
# make the candidates json serializable
for district in election_results:
    winners = election_results[district]["Winners"]
    eliminated = election_results[district]["Eliminated"]
    initial_votes = election_results[district]["Initial Votes"]
    election_results[district]["RepCountTotal"] = vote_map[district][1]
    election_results[district]["VoteThreshold"] = vote_map[district][0] / (1+vote_map[district][1])
    election_results[district]["Winners"] = [(winner[0].name, winner[0].party, winner[1]) for winner in winners]
    election_results[district]["Eliminated"] = [(eliminated[0].name, eliminated[0].party, eliminated[1]) for eliminated in eliminated]
    election_results[district]["Initial Votes"] = [(initial_votes[0].name, initial_votes[0].party, initial_votes[1]) for initial_votes in initial_votes]


In [12]:
import json
# Export the results to a JSON file
with open('ut_election_results.json', 'w') as fp:
    json.dump(election_results, fp)
    
